In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.mft import Mft
from checklist.inv_dir import Inv, Dir
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.perturb import Perturb


In [19]:
from checklist.pred_wrapper import PredictorWrapper

import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
model = model_wrapper.ModelWrapper()
new_pp = PredictorWrapper.wrap_softmax(model.predict_proba)


In [3]:
editor = checklist.editor.Editor()
editor.tg

In [4]:
nlp = spacy.load('en_core_web_sm')

In [135]:
qs = []
labels = []
all_questions = set()
for x in open('/home/marcotcr/datasets/glue/glue_data/QQP/dev.tsv').readlines()[1:]:
    try:
        q1, q2, label = x.strip().split('\t')[3:]
    except:
        print(x)
        continue
    all_questions.add(q1)
    all_questions.add(q2)
    qs.append((q1, q2))
    labels.append(label)
labels = np.array(labels).astype(int)

"	What does the Quran say about homosexuality?	0



In [9]:
import pickle
spacy_map =  pickle.load(open('/home/marcotcr/tmp/processed_qqp.pkl', 'rb'))

In [10]:
parsed_qs = [(spacy_map[q[0]], spacy_map[q[1]]) for q in qs]

## Vocabulary

In [41]:
professions = editor.suggest('{first_name} works as {a:bert}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:bert}.')[:30]
professions = list(set(professions))
# professions

In [36]:
print(', '.join(editor.suggest('{first_name} {last_name} is a good {bert}.')[:30]))

guy, player, writer, example, read, person, actor, friend, hitter, kid, one, pick, character, teacher, student, bet, reader, man, coach, fighter, artist, shooter, choice, name, shot, reporter, story, catch, poet, idea


In [42]:
other_nouns = ['player', 'person', 'friend', 'kid', 'candidate']
nouns = list(set(professions + other_nouns))

In [49]:
print(', '.join(editor.suggest('Is {first_name} {last_name} {a:bert} {noun}?', noun=nouns)[:50]))

effective, actual, active, excellent, appropriate, important, honest, ideal, official, outstanding, independent, adequate, elite, experienced, equal, acceptable, good, American, ethical, successful, authentic, alternative, accurate, real, academic, international, intelligent, interested, unlikely, unusual, early, outside, English, acting, Australian, economic, legitimate, bad, expert, interesting, influential, incompetent, informed, evil, inspired, abusive, professional, better, accredited, aspiring


In [50]:
adjs = ['effective', 'actual', 'American', 'active', 'honest', 'excellent', 'elite', 'acomplished', 'official', 'outstanding', 'experienced', 'independent', 'international', 'aspiring', 'average', 'good', 'amazing', 'exceptional', 'successful', 'accredited', 'English', 'real', 'bad', 'terrible', 'fake', 'unusual', 'influential', 'incompetent']

In [53]:
data = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} {a:adj} {noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    786 (78.6%)

Example fails:
0.8 ('Is Aaron Price a chef?', 'Is Aaron Price an official chef?')
1.0 ('Is Sophia Mitchell a historian?', 'Is Sophia Mitchell a fake historian?')
0.9 ('Is Laura Phillips a friend?', 'Is Laura Phillips an effective friend?')


Is person {a1, a2}?

In [355]:
print(', '.join(editor.suggest('Is John Wayne {bert}?')))
print()
print(', '.join(editor.suggest('Is John Wayne {a:bert}?')[:50]))

dead, gay, alive, right, Dead, real, Jewish, mad, wrong, correct, insane, back, crazy, related, OK, Right, immortal, evil, straight, interested, famous, racist, sane, next, President, alone, lying, relevant, okay, DEAD, bisexual, God, angry, guilty, ok, Alone, Wrong, ready, gone, innocent, King, human, Muslim, Black, assassinated, Batman, here, Real, dying, involved, homosexual, special, Famous, happy, free, murdered, Superman, president, joking, White, Back, Gay, Catholic, cool, safe, resurrected, killed, American, missing, corrupt, cursed, alright, single, Crazy, Mad, Satan, Missing, legit, shot, Evil, finished, worthy, doomed, sincere, aware, autistic, responsible, dangerous, Christian, Cool, Immortal, radioactive, remembered, Next, serious, cheating, different, popular, Legend, listening, now, illegitimate, homophobic, kidding, sick, smart, connected, trustworthy, Innocent, legal

racist, atheist, outlaw, asshole, actor, ape, American, inspiration, idiot, orphan, ancestor, artist, 

In [356]:
adjs_without_overlap = ['dead', 'gay', 'Jewish', 'Christian', 'American', 'mad', 'immortal', 'evil', 'famous', 'racist', 'Muslim', 'white', 'black', 'English', 'autistic', 'Australian', 'trustworthy', 'an atheist', 'an anarchist', 'an inventor', 'Indian', 'Armenian', 'an astronaut', 'an immigrant']

In [461]:
data = editor.template((
    'Is {first_name} {last_name} {adj1}?',
    'Is {first_name} {last_name} {adj2}?',
),
                adj=adjs_without_overlap,
                remove_duplicates=True, 
                nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 959 examples
Test cases:      959
Fails (rate):    10 (1.0%)

Example fails:
0.5 ('Is Rebecca Stewart dead?', 'Is Rebecca Stewart immortal?')
0.6 ('Is Lisa Hill Jewish?', 'Is Lisa Hill an immigrant?')
1.0 ('Is Jordan Cox Jewish?', 'Is Jordan Cox an immigrant?')


person1 and person2 are different by first and last name

In [360]:
data = editor.template((
    'Is {first_name1} {last_name1} {adj}?',
    'Is {first_name2} {last_name2} {adj}?',
),
                adj=adjs_without_overlap,
                remove_duplicates=True, 
                nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 975 examples
Test cases:      975
Fails (rate):    56 (5.7%)

Example fails:
0.8 ('Is Noah King black?', 'Is Samuel Young black?')
0.8 ('Is Daniel James Christian?', 'Is John Scott Christian?')
0.8 ('Is Tiffany Turner American?', 'Is Amy Morgan American?')


person1 and person2 are different by first name only

In [361]:
data = editor.template((
    'Is {first_name} {last_name} {adj}?',
    'Is {first_name2} {last_name} {adj}?',
),
                adj=adjs_without_overlap,
                remove_duplicates=True, 
                nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 984 examples
Test cases:      984
Fails (rate):    94 (9.6%)

Example fails:
0.6 ('Is Tyler Bell an astronaut?', 'Is Sean Bell an astronaut?')
0.8 ('Is Anna Nguyen Muslim?', 'Is Jennifer Nguyen Muslim?')
0.8 ('Is Scott Morris trustworthy?', 'Is Ryan Morris trustworthy?')


person1 and person2 are different by last name only

In [362]:
data = editor.template((
    'Is {first_name} {last_name} {adj}?',
    'Is {first_name} {last_name2} {adj}?',
),
                adj=adjs_without_overlap,
                remove_duplicates=True, 
                nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 992 examples
Test cases:      992
Fails (rate):    76 (7.7%)

Example fails:
0.7 ('Is Natalie Wilson mad?', 'Is Natalie Barnes mad?')
1.0 ('Is Alexis Clark American?', 'Is Alexis Cook American?')
0.9 ('Is Erin Jones Christian?', 'Is Erin Morris Christian?')


## Robustness

In [364]:
def wrap_apply_to_each(fn, both=False, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, str(q2)) for x in fnq1])
        ret.extend([(str(q1), x) for x in fnq2])
        if both:
            ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn
def wrap_apply_to_both(fn, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn

Typos

In [143]:
data = Perturb.perturb(qs, wrap_apply_to_each(Perturb.add_typos), nsamples=500)
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 1500 examples
Test cases:      500
Fails (rate):    108 (21.6%)

Example fails:
1 (1.0) ('Do animals commit suicide?', 'What animals knowingly commit suicide?')
0 (0.0) ('Do animals commit suicide?', 'What anmials knowingly commit suicide?')
0 (0.0) ('Do animals commi tsuicide?', 'What animals knowingly commit suicide?')

1 (1.0) ('What are the functions of the CU and ALU in the CPU of a computer?', 'What is the function of the ALU and CU in a CPU?')
0 (0.1) ('What are the functions of the CU and ALU in hte CPU of a computer?', 'What is the function of the ALU and CU in a CPU?')

1 (1.0) ('How do I log in my Facebook account if you forgot your password?', 'I forgot my Facebook and email password. How can I log into Facebook?')
0 (0.2) ('How do I log in my Facebook account if you forgot your password?', 'I forgot my Facebook and email password. How can I log into Faceboko?')



Contractions

In [368]:
data[2]

[('What are the interesting criminal cases solved?',
  'What are some of the most interesting criminal cases?'),
 ("What're the interesting criminal cases solved?",
  'What are some of the most interesting criminal cases?'),
 ('What are the interesting criminal cases solved?',
  "What're some of the most interesting criminal cases?"),
 ("What're the interesting criminal cases solved?",
  "What're some of the most interesting criminal cases?")]

In [365]:
data = Perturb.perturb(qs, wrap_apply_to_each(Perturb.contractions, both=True), nsamples=500)
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 1506 examples
Test cases:      500
Fails (rate):    8 (1.6%)

Example fails:
1 (0.7) ('What is venture capital financing?', 'What is a venture capital?')
0 (0.3) ("What's venture capital financing?", 'What is a venture capital?')
0 (0.2) ('What is venture capital financing?', "What's a venture capital?")

1 (0.7) ('Is time the fourth dimension?', 'How is time a fourth dimension?')
0 (0.4) ('Is time the fourth dimension?', "How's time a fourth dimension?")

0 (0.5) ('How is viscosity-gravity constant (VGC) determined?', 'How is viscosity-gravity constant (VGC) used?')
1 (0.7) ('How is viscosity-gravity constant (VGC) determined?', "How's viscosity-gravity constant (VGC) used?")
1 (0.9) ("How's viscosity-gravity constant (VGC) determined?", "How's viscosity-gravity constant (VGC) used?")



## Ner

### Change same name, number, location in both

Names

In [166]:
def change_both_wrapper(fn):
    def change_both(qs):
        q1, q2 = qs
        seed = np.random.randint(100)
        c1 = fn(q1, seed=seed, meta=True)
        c2 = fn(q2, seed=seed, meta=True)
        if not c1 or not c2:
            return
        c1, m1 = c1
        c2, m2 = c2
        return [(q1, q2) for q1, q2, m1, m2 in zip(c1, c2, m1, m2) if m1 == m2]
    return change_both
    
data = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_names), nsamples=500)
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 5382 examples
Test cases:      500
Fails (rate):    59 (11.8%)

Example fails:
0 (0.3) ('What were the Jim Crow laws?', 'What were Jim Crow laws in the US? How did they get that name?')
1 (0.6) ('What were the Joseph Jones laws?', 'What were Joseph Jones laws in the US? How did they get that name?')
1 (0.7) ('What were the David Thomas laws?', 'What were David Thomas laws in the US? How did they get that name?')

1 (0.8) ('Why is Quora so completely biased towards Hillary Clinton?', 'Does Quora support Hillary Clinton?')
0 (0.4) ('Why is Quora so completely biased towards Stephanie Ward?', 'Does Quora support Stephanie Ward?')
0 (0.4) ('Why is Quora so completely biased towards Sarah Cox?', 'Does Quora support Sarah Cox?')

1 (1.0) ('Is Donald Trump really a closet liberal?', 'Is Donald Trump a covert operative for the Clintons?')
0 (0.2) ('Is Michael Ortiz really a closet liberal?', 'Is Michael Ortiz a covert operative for the Clintons?')



Locs

In [167]:
data = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_location), nsamples=500)
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 5133 examples
Test cases:      500
Fails (rate):    66 (13.2%)

Example fails:
0 (0.1) ('When will Tibet be part of India?', 'Was Tibet ever a part of India?')
1 (0.5) ('When will Tibet be part of Chad?', 'Was Tibet ever a part of Chad?')
1 (0.6) ('When will Tibet be part of Italy?', 'Was Tibet ever a part of Italy?')

0 (0.0) ('Who want to do business with China?', 'What is your review of Business in China?')
1 (0.8) ('Who want to do business with Slovak Republic?', 'What is your review of Business in Slovak Republic?')

0 (0.0) ('What does India need the most at this moment?', 'What is the Improve India movement? Who leads it?')
1 (0.9) ('What does Macao SAR need the most at this moment?', 'What is the Improve Macao SAR movement? Who leads it?')



Numbers

In [169]:
data = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_number), nsamples=500)
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 4953 examples
Test cases:      500
Fails (rate):    32 (6.4%)

Example fails:
0 (0.3) ('How will you explain the 2008 financial crisis to a teenager?', 'Can you explain the 2008 financial crisis in simple terms? What caused it and how is it still affecting countries?')
1 (0.5) ('How will you explain the 2202 financial crisis to a teenager?', 'Can you explain the 2202 financial crisis in simple terms? What caused it and how is it still affecting countries?')
1 (0.6) ('How will you explain the 1899 financial crisis to a teenager?', 'Can you explain the 1899 financial crisis in simple terms? What caused it and how is it still affecting countries?')

0 (0.1) ('I am preparing for the 2015 CAT and I am very weak in quants. How can I improve my QA score?', 'I am a CAT 2015 aspirant and the test would begin in a month. I have consistently been scoring low in quants and DI because of which I always get very very low scores. I am extremely frustrated now. Is there any way I can improv

### Change name, loc, number in only one where orig prediction is duplicate

Changing only first names

In [162]:
data = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500, first_only=True)
expect_fn = Expect.pairwise(lambda orig, pred, *args: pred == 0)
expect_fn = Expect.slice_pairwise(expect_fn, lambda orig, *args: orig == 1)
test = Dir(data, expect_fn)
test.set_expected_label_print(label=0)
test.run(new_pp)
test.summary(3)

Predicting 23500 examples
Test cases:      1500
After filtering: 593 (39.5%)
Fails (rate):    531 (89.5%)

Example fails:
1 (1.0) ('Why should I vote for Hillary Clinton?', "Why should I vote for Hillary instead of why I shouldn't vote for Trump?")

1 (1.0) ('Is there a good chance Bernie Sanders will run for President in 2020?', 'Do you see Bernie sanders re running in 2020?')

1 (1.0) ('What do you think about the rapist Donald Trump?', 'What do intelligent people think about Donald Trump?')
1 (1.0) ('What do you think about the rapist Donald Trump?', 'What do intelligent people think about Hunter Trump?')
1 (1.0) ('What do you think about the rapist Donald Trump?', 'What do intelligent people think about Lucas Trump?')



Changing first and last names

In [163]:
data = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500)
test = Dir(data, expect_fn)
test.set_expected_label_print(label=0)
test.run(new_pp)
test.summary(3)

Predicting 23550 examples
Test cases:      1500
After filtering: 599 (39.9%)
Fails (rate):    286 (47.7%)

Example fails:
1 (1.0) ('Who is winning the presidential election, Trump or Clinton?', 'Who do you think will win Trump or Clinton?')

1 (1.0) ('What is some historical evidence that Jesus existed?', 'What is the evidence for and against the existence of the historical Jesus?')

1 (1.0) ('Who is a better Person for office Hillary of Donald?', 'Why is Hillary Clinton a better choice than Donald Trump?')
1 (1.0) ('Who is a better Person for office Hillary of Donald?', 'Why is Hillary Clinton a better choice than William Morgan?')



Locs

In [177]:
data = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_location), nsamples=1500)
test = Dir(data, expect_fn)
test.set_expected_label_print(label=0)
test.run(new_pp)
test.summary(3)

Predicting 23260 examples
Test cases:      1500
After filtering: 550 (36.7%)
Fails (rate):    479 (87.1%)

Example fails:
1.0 ('Will banning 500 and 1000 notes can stop the black money?', 'How is black money curbed with the ban of 1000 rupee notes and introducing new 500 and 2000 rupee notes?')
1.0 ('Will banning 488 and 1000 notes can stop the black money?', 'How is black money curbed with the ban of 1000 rupee notes and introducing new 500 and 2000 rupee notes?')
1.0 ('Will banning 500 and 1096 notes can stop the black money?', 'How is black money curbed with the ban of 1000 rupee notes and introducing new 500 and 2000 rupee notes?')

1.0 ('Should people over 99 not be allowed to vote?', 'Should people over 96 not be allowed to vote?')
1.0 ('Should people over 110 not be allowed to vote?', 'Should people over 96 not be allowed to vote?')
1.0 ('Should people over 102 not be allowed to vote?', 'Should people over 96 not be allowed to vote?')

1.0 ('What are your top 5 movies for 2016?'

numbers

In [171]:

data = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_number), nsamples=1500)
test = Dir(data, expect_fn)
test.set_expected_label_print(label=0)
test.run(new_pp)
test.summary(3)

Predicting 23260 examples
Test cases:      1500
After filtering: 550 (36.7%)
Fails (rate):    479 (87.1%)

Example fails:
1 (1.0) ('How do you stop an 8 week Husky puppy from biting my shoes?', 'How do I stop a German Shepherd/Border Collie mix puppy from chewing my shoes?')
1 (1.0) ('How do you stop an 9 week Husky puppy from biting my shoes?', 'How do I stop a German Shepherd/Border Collie mix puppy from chewing my shoes?')
1 (1.0) ('How do you stop an 7 week Husky puppy from biting my shoes?', 'How do I stop a German Shepherd/Border Collie mix puppy from chewing my shoes?')

1 (1.0) ('Is it possible for faithless electors to change the outcome of the 2016 presidential election?', 'According to the law, can faithless electors actually change the result of a presidential election?')

1 (1.0) ('Is World War III coming?', 'Are we getting closer to world war 3?')
1 (1.0) ('Is World War III coming?', 'Are we getting closer to world war 4?')
1 (1.0) ('Is World War III coming?', 'Are we get

In [ ]:
### Change 

### Keep entities, fill in with BERT gibberish

In [221]:
def bert_gibberish(question):
    ents = question.ents
    if not ents:
        return None
    wp = [x.text for x in question if x.tag_ in ['WP', 'WRB', 'WDT']]
    if not wp:
        wp = question[0].text
    else:
        wp = wp[0]
    ents = [x.text for x in ents]
    ents[-1] = ents[-1] + '?'
    template = ' {bert} '.join([wp] + ents)
    gibberish = editor.template(template)[:10]
#     return gibberish
    ret = [(question.text, x) for x in gibberish if question.text != x ]
    return ret
def gibberish_both(qs):
    q1, q2 = qs
    ret = []
    x1 = bert_gibberish(q1)
    if x1:
        ret.extend(x1)
    x2 = bert_gibberish(q2)
    if x2:
        ret.extend(x2)
    return ret
#     ret =  [(question.text, x[1]) for x in tg.fill_in_between([wp] + ents) if question.text != x[1]]
#     ret = [x for x in ret if len(x[1].split()) < len(x[0].split()) -4][:5]
#     return ret 

In [251]:
data = Perturb.perturb(parsed_qs, gibberish_both, nsamples=500)
expect_false = Expect.pairwise(lambda orig, pred, *args: pred == 0)
test = Dir(data, expect_false)
test.set_expected_label_print(label=0)
test.run(new_pp)
test.summary(3)

Predicting 8785 examples
Test cases:      500
Fails (rate):    162 (32.4%)

Example fails:
0 (0.0) ('Is it theoretically possible to travel a light year in under one year due to length contraction and other relativistic effects?', "Is Run Ze Cao's falsification of Einstein's relativity valid?")
1 (1.0) ('Is it theoretically possible to travel a light year in under one year due to length contraction and other relativistic effects?', 'Is it a light year … under one year?')
1 (1.0) ('Is it theoretically possible to travel a light year in under one year due to length contraction and other relativistic effects?', 'Is it a light year ... under one year?')

0 (0.0) ('How is the placement of the MBA students at BIT Mesra?', 'What is the average package of placement in the civil branch of BIT Mesra?')
1 (0.9) ('How is the placement of the MBA students at BIT Mesra?', 'How Do MBA I BIT Mesra?')
1 (0.6) ('How is the placement of the MBA students at BIT Mesra?', 'How does MBA B BIT Mesra?')

0 (0.

In [260]:
test.summary(3)

Test cases:      500
Fails (rate):    162 (32.4%)

Example fails:
0 (0.0) ('What happens at the end of the 2002 movie I Not Stupid?', 'What happens at the end of the 2002 movie The Hours?')
1 (1.0) ('What happens at the end of the 2002 movie I Not Stupid?', 'What happened the end of the 2002?')
1 (0.8) ('What happens at the end of the 2002 movie I Not Stupid?', 'What at the end of the 2002?')

1 (1.0) ('Eighth Generation Consoles: What are the differences between XBox One and Playstation 4 and which do you prefer?', 'Which is Better, the XBOX One or PlayStation 4?')
1 (1.0) ('Which is Better, the XBOX One or PlayStation 4?', "Which 's Better - PlayStation 4?")
1 (1.0) ('Eighth Generation Consoles: What are the differences between XBox One and Playstation 4 and which do you prefer?', 'What Reviews Eighth Sony XBox One?')

1 (1.0) ('What do software developers miss when switching from Linux to Mac OS?', 'What are the main features a Linux (Ubuntu) user would miss when moving to Mac OS X?

## Temporal

Is != used to be

In [266]:
data = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Did {first_name} {last_name} use to be {a:noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    706 (70.6%)

Example fails:
0.9 ('Is Jessica Miller a secretary?', 'Did Jessica Miller use to be a secretary?')
0.9 ('Is Katherine Ward a programmer?', 'Did Katherine Ward use to be a programmer?')
0.7 ('Is Rebecca Lewis a teacher?', 'Did Rebecca Lewis use to be a teacher?')


Is != becoming

In [374]:
data = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} becoming {a:noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    192 (19.2%)

Example fails:
0.6 ('Is Erin Fisher an editor?', 'Is Erin Fisher becoming an editor?')
0.7 ('Is Nathan Collins an organizer?', 'Is Nathan Collins becoming an organizer?')
0.8 ('Is Taylor Gomez a kid?', 'Is Taylor Gomez becoming a kid?')


Before != after

In [371]:
data = editor.template((
    'What was {first_name} {last_name}\'s life before becoming {a:noun}?',
    'What was {first_name} {last_name}\'s life after becoming {a:noun}?'
),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    1000 (100.0%)

Example fails:
1.0 ("What was Alyssa Nguyen's life before becoming a secretary?", "What was Alyssa Nguyen's life after becoming a secretary?")
1.0 ("What was Samuel Reyes's life before becoming an administrator?", "What was Samuel Reyes's life after becoming an administrator?")
1.0 ("What was Thomas Flores's life before becoming a reporter?", "What was Thomas Flores's life after becoming a reporter?")


In [269]:
print(', '.join(editor.suggest('Is is {bert} to eat after 10pm?')))

it, normal, OK, there, safe, best, ok, okay, acceptable, reasonable, permissible, legal, proper, wrong, healthy, good, enough, better, important, appropriate, left, allowed, possible, dinner, vegetarian, what, recommended, going, food, illegal, supposed, smart, unhealthy, hard, popular, hot, necessary, vegan, typical, not, free, right, required, fun, time, meant, something, one, kosher, customary, fair, mandatory, available, common, cheaper, alright, easier, suitable, feasible, polite, difficult, cool, certain, sensible, affordable, advisable, fine, fashionable, like, nice, unsafe, public, wise, decent, sufficient, harder, advised, traditional, usual, healthier, anyone, permitted, eating, easy, life, trendy, someone, forbidden, lawful, expensive, safer, new, everyone, compulsory, dangerous, worse, needed, realistic, temptation, essential, adequate, impossible, hungry, rude, risky, considered, prudent, taboo, different, tempting


In [272]:
mid = ['normal', 'ok', 'safe', 'dangerous', 'acceptable', 'reasonable', 'proper', 'wrong', 'healthy', 'important']

In [273]:
print(', '.join(editor.suggest('Is is {mid} to {bert} after 10pm?', mid=mid)))

drink, sleep, drive, work, eat, leave, smoke, go, stay, travel, visit, vote, call, walk, read, party, talk, do, retire, fish, post, play, exercise, watch, be, celebrate, write, return, study, stop, disappear, continue, move, shop, text, strike, tweet, start, live, run, speak, relax, dance, act, fly, swim, die, enter, gamble, report, operate, bed, meet, kill, remain, wait, gather, attend, come, feed, cook, rise, film, finish, blog, close, paint, change, linger, quit, pray, ask, answer, pee, park, disturb, propose, dress, sit, withdraw, rest, protest, avoid, skate, check, stand, crash, marry, see, fight, open, vanish, exit, have, commute, chat, spend, cycle, perform, look, vape, arrive, cross, phone, proceed, buy, breathe, kiss, demonstrate, publish, pass, behave, respond, bar, complain, hang, function, dinner, exist, nap, practice, begin, end, consume, resign, use, ride, bike, bet, think, cry, shower, wander, escape, comment, approach, refuse, roam, know, dive, bicycle, emerge, explore,

In [274]:
activity = ['drink', 'sleep', 'drive', 'work', 'eat', 'smoke', 'walk', 'read', 'party', 'talk', 'exercise', 'celebrate', 'text', 'tweet', 'run', 'dance', 'swim', 'cook', 'pray', 'pee', 'rest']

In [275]:
data = editor.template(('Is it {mid} to {activity} before {hour}{ampm}?','Is it {mid} to {activity} after {hour}{ampm}?'),
                activity=activity,
                mid=mid,
                hour=[str(x) for x in range(1, 12)],
                ampm=['am', 'pm'],
                remove_duplicates=True, 
                nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    971 (97.1%)

Example fails:
0.9 ('Is it dangerous to dance before 9pm?', 'Is it dangerous to dance after 9pm?')
0.9 ('Is it acceptable to party before 1am?', 'Is it acceptable to party after 1am?')
0.9 ('Is it acceptable to drink before 2pm?', 'Is it acceptable to drink after 2pm?')


## Negation

In [277]:
mid2 = mid + ['legal', 'awkward', 'socially acceptable']

In [282]:
print(', '.join(editor.suggest('Is it {mid} to {bert} in {country}?', mid=mid2)[:100]))

live, work, vote, stay, marry, be, drive, study, travel, eat, protest, campaign, squat, fight, die, fly, shop, speak, gamble, demonstrate, visit, hunt, remain, pray, sleep, smoke, serve, fish, exist, write, settle, kill, murder, reside, meet, swim, invest, operate, move, pee, march, perform, strike, play, act, report, worship, rape, arrive, gather, drink, volunteer, sue, propose, fire, surf, shoot, dance, land, party, hate, practice, race, vacation, hide, cook, paint, camp, train, preach, teach, burn, spy, call, sit, build, stand, ask, search, walk, buy, ski, survive, publish, appear, think, talk, park, continue, film, lie, exercise, gay, holiday, divorce, start, believe, look, discriminate, migrate


In [285]:
things = ['work', 'vote', 'travel', 'marry', 'drive', 'study', 'protest', 'campaign', 'fight', 'gamble', 'hunt', 'pray', 'smoke', 'fish', 'murder', 'invest', 'pee', 'march', 'worship', 'rape', 'volunteer', 'surf', 'shoot', 'dance', 'camp', 'preach', 'spy', 'be gay', 'lie', 'divorce', 'discriminate']

In [287]:
data = editor.template(('Is it {mid} to {activity} in {country}?','Is it {mid} not to {activity} in {country}?'),
                activity=things,
                mid=mid2,
                remove_duplicates=True, 
                nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    238 (23.8%)

Example fails:
0.9 ('Is it dangerous to smoke in Azerbaijan?', 'Is it dangerous not to smoke in Azerbaijan?')
0.9 ('Is it reasonable to pee in Comoros?', 'Is it reasonable not to pee in Comoros?')
0.6 ('Is it wrong to travel in Myanmar?', 'Is it wrong not to travel in Myanmar?')


In [396]:
data = editor.template((
    'What are things {a:noun} should worry about?',
    'What are things {a:noun} should not worry about?',
),
                noun=nouns,
                remove_duplicates=True, 
                nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)


## Coref

In [290]:
data = editor.template(
    (
        'If {male} and {female} were alone, do you think he would reject her?',
        'If {male} and {female} were alone, do you think she would reject him?',
    ),
    remove_duplicates=True, 
    nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    702 (70.2%)

Example fails:
0.8 ('If Bryan and Andrea were alone, do you think he would reject her?', 'If Bryan and Andrea were alone, do you think she would reject him?')
0.7 ('If Charles and Andrea were alone, do you think he would reject her?', 'If Charles and Andrea were alone, do you think she would reject him?')
0.8 ('If Adam and Ava were alone, do you think he would reject her?', 'If Adam and Ava were alone, do you think she would reject him?')


In [294]:
data = editor.template(
    [(
        'If {male} and {female} were married, would his family be happy?',
        'If {male} and {female} were married, would {female}\'s family be happy?',
    ),(
        'If {male} and {female} were married, would her family be happy?',
        'If {male} and {female} were married, would {male}\'s family be happy?',
    ),
    ]
        ,
    unroll=True,
    remove_duplicates=True, 
    nsamples=1000)
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 2000 examples
Test cases:      2000
Fails (rate):    1979 (99.0%)

Example fails:
1.0 ('If Daniel and Melissa were married, would his family be happy?', "If Daniel and Melissa were married, would Melissa's family be happy?")
0.9 ('If Jack and Holly were married, would his family be happy?', "If Jack and Holly were married, would Holly's family be happy?")
0.8 ('If Logan and Amy were married, would her family be happy?', "If Logan and Amy were married, would Logan's family be happy?")


## SRL

In [295]:
print(', '.join(editor.suggest('Who is the best {bert} in the world?')))

chef, pizza, boxer, player, footballer, athlete, rapper, actor, singer, cook, magician, robot, coach, beer, cyclist, wrestler, drummer, musician, quarterback, hacker, dog, baker, fighter, journalist, restaurant, steak, teacher, doctor, gamer, band, shooter, DJ, football, person, horse, photographer, driver, fisherman, burger, coffee, lawyer, writer, food, dancer, student, wine, artist, man, surgeon, comedian, trainer, VPN, vegetarian, programmer, team, game, vegan, guitarist, goalkeeper, server, guy, cricket, sport, engineer, dentist, waiter, cheese, bartender, fish, chicken, goalie, car, manager, computer, referee, hunter, guitar, basketball, tennis, astronaut, mathematician, sandwich, blogger, accountant, TV, soldier, sniper, job, runner, barbecue, chocolate, scientist, friend, cat, gun, pitcher, judge, AI, philosopher, butcher, painter, golf, camel, translator, piano, worker, farmer, conductor, politician, bike


In [298]:
thing = ['chef', 'boxer', 'player', 'footballer', 'athlete', 'rapper', 'actor', 'singer', 'cook', 'magician', 'coach', 'cyclist', 'wrestler', 'drummer', 'musician', 'quarterback', 'hacker', 'baker', 'fighter', 'journalist', 'teacher', 'doctor', 'gamer', 'husband', 'DJ', 'person', 'man', 'woman', 'surgeon', 'comedian', 'trainer', 'programmer', 'guitarist', 'goalkeeper']

In [300]:
print(', '.join(editor.suggest('Who do {bert} think is the the best {thing} in the world?', thing=thing)))

you, You, YOU, people, we, they, I, readers, u, guys, your, others, fans, ya, experts, scientists, Americans, students, men, voters, authors, some, the, friends, everyone, conservatives, women, Canadians, analysts, critics, he, players, judges, audiences, artists, researchers, most, folks, liberals, all, historians, Australians, celebrities, members, journalists, supporters, respondents, viewers, teachers, i, participants, Republicans, coaches, parents, editors, many, comedians, writers, kids, U, economists, reporters, consumers, veterans, doctors, gamers, anyone, users, independents, individuals, millennials, candidates, philosophers, athletes, reviewers, politicians, commentators, pundits, atheists, each, children, yo, scholars, psychologists, archaeologists, Christians, feminists, agents, ye, adults, pros, seniors, ladies, teenagers, investigators, not, both, listeners, Democrats, leaders, humans, YOUR, police, academics, contestants, stars, those, astronomers, guests, musicians, an

In [301]:
subjects = ['you', 'people', 'readers', 'guys', 'fans', 'experts', 'scientists', 'Americans', 'students', 'men', 'voters', 'authors', 'conservatives', 'women', 'Canadians', 'analysts', 'critics', 'judges', 'artists', 'researchers', 'liberals', 'historians', 'Australians', 'journalists', 'Republicans', 'coaches', 'parents', 'kids', 'economists', 'reporters', 'consumers', 'veterans', 'doctors']

In [303]:
print(', '.join(editor.suggest('Who do {subjects} think is the the {bert} {thing} in the world?', thing=thing, subjects=subjects)[:50]))

best, greatest, worst, top, smartest, strongest, biggest, toughest, finest, fastest, happiest, coolest, deadliest, hottest, hardest, better, safest, richest, oldest, leading, great, youngest, brightest, busiest, largest, superior, favorite, premier, ultimate, elite, most, dominant, Greatest, easiest, highest, favourite, perfect, BEST, wealthiest, weakest, newest, Best, only, outstanding, smallest, professional, right, next, good, heaviest


In [304]:
best = ['best', 'greatest', 'worst', 'top', 'smartest', 'strongest', 'finests', 'happiest', 'coolest', 'richest', 'leading', 'brightest', 'premier', 'ultimate', 'dominant']

In [305]:
data = editor.template((
    'Who do {subjects} think is the {best} {thing} in the world?',
    'Who is the {best} {thing} in the world according to {subjects}?'
),
    subjects=subjects,
    best=best,
    thing=thing,
    remove_duplicates=True, 
    nsamples=1000)
test = Mft(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    135 (13.5%)

Example fails:
0.4 ('Who do students think is the greatest singer in the world?', 'Who is the greatest singer in the world according to students?')
0.0 ('Who do men think is the worst person in the world?', 'Who is the worst person in the world according to men?')
0.3 ('Who do men think is the worst programmer in the world?', 'Who is the worst programmer in the world according to men?')


In [319]:
print(', '.join([str(x) for x in editor.suggest('Are {bert} smaller than {a}?', a=['bananas', 'dogs', 'cars', 'cats', 'elephants'])][:100]))

dogs, cats, humans, people, pigs, chickens, you, mice, birds, cows, sheep, fish, bees, snakes, elephants, we, bats, robots, ants, spiders, babies, puppies, insects, trees, animals, they, dolphins, butterflies, flies, whales, mosquitoes, plants, horses, cars, dinosaurs, goats, rats, bears, men, monkeys, rabbits, lions, pets, children, kittens, frogs, girls, turtles, ducks, bugs, kids, crabs, computers, tigers, worms, toys, things, sharks, primates, beetles, wolves, chimpanzees, cattle, apes, deer, reptiles, rodents, mammals, houses, diamonds, seals, bicycles, ponies, potatoes, drones, mushrooms, dragons, apples, boys, us, flowers, carrots, women, dolls, ticks, balls, coins, stones, brains, calves, Indians, dwarves, trolls, pedestrians, trucks, bulls, vampires, blacks, Americans, bikes


In [325]:
things = editor.suggest('Are {bert} smaller than {a}?',a=['bananas', 'dogs', 'cars', 'cats', 'elephants'] )[:100]

In [320]:
print(', '.join([str(x) for x in editor.suggest('Are {a} {bert} than {a2}?', a=things, a2=things)][:100]))

better, rather, more, worse, easier, harder, cheaper, safer, less, different, faster, greater, smarter, stronger, cleaner, other, bigger, tougher, healthier, quicker, stranger, cooler, higher, nicer, important, louder, deeper, er, longer, simpler, clearer, slower, wiser, hotter, ier, larger, closer, sooner, happier, preferred, preferable, legal, richer, dead, lower, work, superior, older, warmer, thicker, criminal, necessary, popular, lighter, dangerous, murder, useful, younger, differently, further, orse, darker, right, smaller, fun, essential, der, broader, effective, help, wider, shorter, productive, quieter, r, real, weaker, harsher, lesser, Rather, valuable, death, fatal, harmful, hurt, free, prefer, powerful, ger, cheat, killer, ter, colder, crime, rape, lier, art, cruel, political, heavier


In [321]:
comp = ['better', 'worse', 'cheaper', 'bigger', 'louder', 'longer', 'larger', 'smaller', 'warmer', 'colder', 'thicker', 'lighter', 'heavier']

Order doesn't matter for comparison

In [334]:
data = editor.template((
    'Are {t1} {comp} than {t2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'What is {comp}, {t2} or {t1}?'
),
    t = things,
    comp = comp,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = Mft(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 993 examples
Test cases:      993
Fails (rate):    987 (99.4%)

Example fails:
0.0 ('Are sheep heavier than diamonds?', 'What is heavier, diamonds or sheep?')
0.0 ('Are girls bigger than trees?', 'What is bigger, trees or girls?')
0.0 ('Are cows bigger than chickens?', 'What is bigger, chickens or cows?')


Order matters for asymetric relations

In [340]:
# editor.template('Is {first_name1} {bert} {first_name2}?', remove_duplicates=True)[:100]

In [344]:
print(', '.join(editor.suggest('Is {first_name1} {bert} to {first_name2}?', remove_duplicates=True)[:100]))
print()
print(', '.join(editor.suggest('Is {first_name1} {bert} {first_name2}?', remove_duplicates=True)[:100]))

related, married, close, engaged, attracted, talking, lying, closer, connected, speaking, going, linked, loyal, mean, faithful, similar, important, listening, true, next, committed, dead, writing, tied, up, proposing, referring, responding, bound, on, hostile, lied, turning, gay, returning, closest, straight, new, opposed, right, proposed, coming, indebted, back, real, abusive, truthful, known, allergic, attached, crazy, nice, good, talk, reacting, Married, dangerous, addicted, available, attractive, supposed, fair, alive, happening, friendly, getting, gone, special, happy, devoted, nicer, relevant, out, wrong, kind, open, bisexual, decent, used, drawn, kin, entitled, guilty, single, insane, cruel, wed, cold, father, serious, clinging, off, moving, acceptable, Loyal, superior, pregnant, equal, unfair, helpful

or, and, a, really, an, dating, with, actually, &, /, like, still, to, Really, not, marrying, married, real, Dating, ,, after, from, indeed, the, leaving, even, his, killing, bor

In [346]:
symmetric = ['dating', 'married to', 'close to', 'engaged to', 'connected to', 'married to', 'friends with', 'related to', 'an acquaintance of']

In [347]:
data = editor.template((
    'Is {first_name1} {s} {first_name2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'Is {first_name2} {s} {first_name1}?',
),
    s = symmetric,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = Mft(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 987 examples
Test cases:      987
Fails (rate):    786 (79.6%)

Example fails:
0.2 ('Is Katherine an acquaintance of James?', 'Is James an acquaintance of Katherine?')
0.1 ('Is Sara an acquaintance of John?', 'Is John an acquaintance of Sara?')
0.0 ('Is Laura related to Shannon?', 'Is Shannon related to Laura?')


In [ ]:
asymmetric = ['hurting', 'lying to', 'loyal to', 'faithful to', 'proposing to', 'indebted to', 'abusive to', 'using', 'expecting', 'beating', 'punching', 'raising', 'poisoning', 'protecting', 'kidnapping']

In [348]:
data = editor.template((
    'Is {first_name1} {s} {first_name2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'Is {first_name2} {s} {first_name1}?',
),
    s = asymmetric,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 995 examples
Test cases:      995
Fails (rate):    273 (27.4%)

Example fails:
0.7 ('Is Timothy beating Michael?', 'Is Michael beating Timothy?')
0.6 ('Is Ethan proposing to Charles?', 'Is Charles proposing to Ethan?')
0.9 ('Is Justin using Jonathan?', 'Is Jonathan using Justin?')


More traditional SRL, now possible with new interface (was hard before)

In [399]:
print(', '.join(editor.suggest('Did John buy the {bert}?', remove_duplicates=True)[:100]))

farm, stake, house, property, company, land, rights, ticket, papers, newspaper, book, island, estate, ranch, boat, horse, paper, shares, tickets, business, gun, books, newspapers, yacht, team, ship, stock, idea, plot, contract, Bible, phone, franchise, shotgun, store, horses, cattle, church, place, game, factory, castle, club, painting, bank, tract, rifle, manuscript, car, campaign, Ark, slaves, plane, school, money, guns, sword, plantation, beer, trust, election, film, ring, building, station, twins, deal, loan, time, tractor, home, letter, IRA, plan, title, watch, story, letters, rest, cows, piece, diary, brewery, charter, diamond, mortgage, hospital, site, post, operation, insurance, railroad, cow, coins, Dell, case, children, wine, telephone, dog


In [401]:
obj = ['farm', 'house', 'property', 'company', 'land', 'ticket', 'newspaper', 'book', 'island', 'estate', 'ranch', 'boat', 'horse', 'paper', 'business', 'gun', 'game', 'factory', 'castle', 'painting', 'rifle', 'car', 'school', 'building']

In [402]:
print(', '.join(editor.suggest('Did John {bert} the {obj}?', obj=obj, remove_duplicates=True)[:100]))

buy, sell, leave, own, take, get, keep, see, want, lose, have, purchase, destroy, inherit, find, use, need, receive, return, like, enjoy, abandon, manage, remember, finish, run, miss, move, handle, seize, build, steal, acquire, rebuild, save, afford, hold, sign, win, recover, maintain, enter, control, burn, know, break, raid, reclaim, bought, start, rent, close, restore, sold, survive, claim, give, name, join, notice, deliver, sue, rob, make, visit, clear, in, left, retain, accept, bring, fix, operate, mention, quit, watch, remove, tour, secure, call, flee, found, recommend, touch, on, open, repair, complete, share, choose, love, mortgage, lead, escape, inherited, mind, recall, develop, pursue, land


In [457]:
import pattern
import pattern.en
verbs = ['buy', 'purchase', 'sell', 'leave', 'own', 'take', 'keep', 'want', 'lose', 'destroy', 'inherit', 'find', 'use', 'need', 'receive', 'return', 'like', 'enjoy', 'abandon', 'manage', 'remember', 'miss', 'move', 'seize', 'steal']
a = pattern.en.tenses('stolen')[0]
verbs = [(v, pattern.en.conjugate(v, *a)) for v in verbs]
verbs[3] = ('leave', 'left')
verbs

[('buy', 'bought'),
 ('purchase', 'purchased'),
 ('sell', 'sold'),
 ('leave', 'left'),
 ('own', 'owned'),
 ('take', 'taken'),
 ('keep', 'kept'),
 ('want', 'wanted'),
 ('lose', 'lost'),
 ('destroy', 'destroyed'),
 ('inherit', 'inherited'),
 ('find', 'found'),
 ('use', 'used'),
 ('need', 'needed'),
 ('receive', 'received'),
 ('return', 'returned'),
 ('like', 'liked'),
 ('enjoy', 'enjoyed'),
 ('abandon', 'abandoned'),
 ('manage', 'managed'),
 ('remember', 'remembered'),
 ('miss', 'missed'),
 ('move', 'moved'),
 ('seize', 'seized'),
 ('steal', 'stolen')]

In [447]:
data = editor.template((
    'Did {first_name} {verb[0]} the {obj}?',
    'Was the {obj} {verb[1]} by {first_name}?'
),
    verb=verbs,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = Mft(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    112 (11.2%)

Example fails:
0.4 ('Did Richard abandon the island?', 'Was the island abandoned by Richard?')
0.3 ('Did Aaron use the estate?', 'Was the estate used by Aaron?')
0.0 ('Did Daniel miss the land?', 'Was the land missed by Daniel?')


In [456]:
data = editor.template((
    'Did {first_name} {verb[0]} the {obj}?',
    'Was {first_name} {verb[1]} by the {obj}?'
),
    verb=verbs,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    953 (95.3%)

Example fails:
0.9 ('Did Justin find the business?', 'Was Justin found by the business?')
0.9 ('Did Mark keep the paper?', 'Was Mark kept by the paper?')
0.9 ('Did Michael remember the land?', 'Was Michael remembered by the land?')


With people

In [448]:
print(', '.join(editor.suggest('Does {first_name} {bert} {first_name2}?', remove_duplicates=True)[:100]))

know, love, kill, marry, hate, like, and, or, remember, want, see, date, meet, miss, have, Know, need, Love, recognize, find, tell, trust, choose, deserve, understand, mean, forgive, blame, mention, beat, dislike, prefer, get, follow, notice, murder, hurt, bother, Remember, leave, kidnap, ask, own, survive, call, support, visit, /, resemble, take, kiss, hit, save, Like, recognise, become, help, believe, accept, reject, attack, ?, shoot, ,, eat, hire, Want, clone, be, &, pick, contact, fear, Hate, adopt, respect, Kill, crush, use, divorce, rape, catch, replace, KILL, scare, join, pursue, threaten, play, confront, likes, met, lose, married, WANT, stalk, regret, include, despise, resent


In [453]:
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('stolen')[0]
pverb = [(v, pattern.en.conjugate(v, *a)) for v in pverb]
data = editor.template((
    'Does {first_name} {verb[0]} {first_name2}?',
    'Is {first_name2} {verb[1]} by {first_name}?',
),
    verb=pverb,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = Mft(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 989 examples
Test cases:      989
Fails (rate):    665 (67.2%)

Example fails:
0.1 ('Does Erin love Kyle?', 'Is Kyle loved by Erin?')
0.0 ('Does Erin hurt Alexis?', 'Is Alexis hurt by Erin?')
0.4 ('Does Scott blame Jennifer?', 'Is Jennifer blamed by Scott?')


In [454]:
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('stolen')[0]
pverb = [(v, pattern.en.conjugate(v, *a)) for v in pverb]
data = editor.template((
    'Does {first_name} {verb[0]} {first_name2}?',
    'Is {first_name} {verb[1]} by {first_name2}?',
),
    verb=pverb,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 991 examples
Test cases:      991
Fails (rate):    972 (98.1%)

Example fails:
0.9 ('Does Joseph like Laura?', 'Is Joseph liked by Laura?')
1.0 ('Does Ethan trust Timothy?', 'Is Ethan trusted by Timothy?')
0.9 ('Does Austin trust Anna?', 'Is Austin trusted by Anna?')
